In [1]:
#7.3 Final Data Mining after Search for Patterns
import pandas as pd
hotel1=pd.read_csv("./hotel_final.csv")

In [2]:
#7.3.1 Adding some attributes
hotel1 ['have_adults']=hotel1.apply(lambda x:0 if x.adults==0 else 1,axis=1)
hotel1 ['have_children']=hotel1.apply(lambda x:0 if x.children==0 else 1,axis=1)     
hotel1 ['have_babies']=hotel1.apply(lambda x:0 if x.babies==0 else 1,axis=1)
hotel1 ['changes']=hotel1.apply(lambda x:0 if x.booking_changes==0 else 1,axis=1)
hotel1 ['have_parking']=hotel1.apply(lambda x:0 if x.required_car_parking_spaces==0 else 1,axis=1)


In [3]:
hotel1.to_csv('./hotel_final_S.csv')

In [4]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree_methods').getOrCreate()

In [5]:
Hotel_final=spark.read.csv('./hotel_final_S.csv',inferSchema=True,header=True)

In [6]:
#7.3.1 Deleting some attributes
Hotel_final=Hotel_final.drop("days_in_waiting_list")
Hotel_final=Hotel_final.drop("required_car_parking_spaces")
Hotel_final=Hotel_final.drop("stays_in_week_nights")
Hotel_final=Hotel_final.drop("adults")
Hotel_final=Hotel_final.drop("children")
Hotel_final=Hotel_final.drop("babies")
Hotel_final=Hotel_final.drop("booking_changes")
Hotel_final=Hotel_final.drop("Unnamed: 0")
Hotel_final=Hotel_final.drop("Unnamed: 0.1")
Hotel_final=Hotel_final.drop("Unnamed: 0.1.1")
Hotel_final=Hotel_final.drop("_c0")
Hotel_final=Hotel_final.drop("stays_in_weekend_nights")
Hotel_final=Hotel_final.drop("reserved_room_type")
Hotel_final=Hotel_final.drop("assigned_room_type")
Hotel_final.printSchema()

root
 |-- hotel: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- meal: integer (nullable = true)
 |-- market_segment: integer (nullable = true)
 |-- distribution_channel: integer (nullable = true)
 |-- deposit_type: integer (nullable = true)
 |-- customer_type: integer (nullable = true)
 |-- room_change: integer (nullable = true)
 |-- have_requests: integer (nullable = true)
 |-- have_adults: integer (nullable = true)
 |-- have_children: integer (nullable = true)
 |-- have_babies: integer (nullable = true)
 |-- changes: integer (nullable = true)
 |-- have_parking: integer (nullable = true)



In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline
assembler = VectorAssembler(
    inputCols=["hotel", "arrival_date_year","meal","market_segment","distribution_channel","deposit_type","customer_type","room_change","have_adults","have_children","have_babies","changes","have_parking"],
    outputCol="features")

In [10]:
output = assembler.transform(Hotel_final)
output.printSchema()

root
 |-- hotel: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- meal: integer (nullable = true)
 |-- market_segment: integer (nullable = true)
 |-- distribution_channel: integer (nullable = true)
 |-- deposit_type: integer (nullable = true)
 |-- customer_type: integer (nullable = true)
 |-- room_change: integer (nullable = true)
 |-- have_requests: integer (nullable = true)
 |-- have_adults: integer (nullable = true)
 |-- have_children: integer (nullable = true)
 |-- have_babies: integer (nullable = true)
 |-- changes: integer (nullable = true)
 |-- have_parking: integer (nullable = true)
 |-- features: vector (nullable = true)



In [11]:
final_data=output.select("features",'have_requests')

In [12]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [13]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

In [46]:
dtc = DecisionTreeClassifier(labelCol='have_requests',featuresCol='features')
rfc = RandomForestClassifier(labelCol='have_requests',featuresCol='features',numTrees=10,maxDepth=3)
gbt = GBTClassifier(labelCol='have_requests',featuresCol='features')

In [47]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [48]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

In [49]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [50]:
my_binary_eval = BinaryClassificationEvaluator(labelCol = 'have_requests')

In [51]:
print("DTC")
print(my_binary_eval.evaluate(dtc_predictions))

DTC
0.6675304753484651


In [52]:
print("RFC")
print(my_binary_eval.evaluate(rfc_predictions))

RFC
0.7462256138004867


In [53]:
my_binary_gbt_eval = BinaryClassificationEvaluator(labelCol='have_requests', rawPredictionCol='prediction')
print("GBT")
print(my_binary_gbt_eval.evaluate(gbt_predictions))

GBT
0.7226231439652905


In [54]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [55]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol="have_requests", predictionCol="prediction", metricName="accuracy")

In [56]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [57]:
print("Here are the results!")
print('-'*40)
print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*40)
print('A random forest ensemble has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*40)
print('An ensemble using GBT has an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
----------------------------------------
A single decision tree has an accuracy of: 71.89%
----------------------------------------
A random forest ensemble has an accuracy of: 71.41%
----------------------------------------
An ensemble using GBT has an accuracy of: 72.27%


In [58]:
random_forest_model=rfc.fit(train_data)
random_forest_predictions=random_forest_model.transform(test_data)

In [59]:
print(random_forest_model.toDebugString)

RandomForestClassificationModel (uid=rfc_a74bb6967fc9) with 10 trees
  Tree 0 (weight 1.0):
    If (feature 3 <= 1.0)
     If (feature 6 <= 2.0)
      If (feature 9 <= 0.0)
       Predict: 1.0
      Else (feature 9 > 0.0)
       Predict: 1.0
     Else (feature 6 > 2.0)
      If (feature 1 <= 2016.0)
       Predict: 1.0
      Else (feature 1 > 2016.0)
       Predict: 1.0
    Else (feature 3 > 1.0)
     If (feature 12 <= 0.0)
      If (feature 1 <= 2015.0)
       Predict: 0.0
      Else (feature 1 > 2015.0)
       Predict: 0.0
     Else (feature 12 > 0.0)
      If (feature 9 <= 0.0)
       Predict: 0.0
      Else (feature 9 > 0.0)
       Predict: 1.0
  Tree 1 (weight 1.0):
    If (feature 6 <= 3.0)
     If (feature 4 <= 1.0)
      If (feature 9 <= 0.0)
       Predict: 1.0
      Else (feature 9 > 0.0)
       Predict: 1.0
     Else (feature 4 > 1.0)
      If (feature 1 <= 2016.0)
       Predict: 0.0
      Else (feature 1 > 2016.0)
       Predict: 0.0
    Else (feature 6 > 3.0)
     If (fea

In [60]:
from sklearn.metrics import classification_report, confusion_matrix

In [61]:
y_true=rfc_predictions.select(['have_requests']).collect()
y_pred=rfc_predictions.select(['prediction']).collect()
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.69      0.74      0.72     11010
           1       0.74      0.69      0.71     11582

    accuracy                           0.71     22592
   macro avg       0.72      0.71      0.71     22592
weighted avg       0.72      0.71      0.71     22592

